In [17]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset, ConcatDataset
from torchvision import transforms, datasets
from PIL import Image
import os
import zipfile
import torch.optim as optim
import torch.nn as nn
import numpy as np
from sklearn.model_selection import KFold
import random
from torch.cuda.amp import autocast, GradScaler
from torch.profiler import profile, record_function, ProfilerActivity
import torch.cuda.nvtx as nvtx
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
scaler = GradScaler()

C:\Users\Sai\AppData\Local\Temp\ipykernel_38372\4040221952.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [18]:
print(f"Using device: {device}")

Using device: cuda:0


In [19]:
# Load experimental data
def load_all_experimental_data(test_digits_folder):
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []
    participant_data = {}

    transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    for filename in os.listdir(test_digits_folder):
        if filename.endswith('.zip') and filename.startswith('experiment_results_participant'):
            participant_number = int(filename.split('participant')[1].split('.')[0])
            zip_filepath = os.path.join(test_digits_folder, filename)

            participant_train_images = []
            participant_train_labels = []
            participant_test_images = []
            participant_test_labels = []

            with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                for img_filename in zip_ref.namelist():
                    if img_filename.endswith('.png'):
                        with zip_ref.open(img_filename) as file:
                            img = Image.open(file).convert('L')
                            img_tensor = transform(img)
                            
                            digit = int(img_filename.split('_')[0])
                            
                            if 'composite' in img_filename:
                                test_images.append(img_tensor)
                                test_labels.append(digit)
                                participant_test_images.append(img_tensor)
                                participant_test_labels.append(digit)
                            else:
                                train_images.append(img_tensor)
                                train_labels.append(digit)
                                participant_train_images.append(img_tensor)
                                participant_train_labels.append(digit)

            participant_data[participant_number] = {
                'train': (torch.stack(participant_train_images), torch.tensor(participant_train_labels)),
                'test': (torch.stack(participant_test_images), torch.tensor(participant_test_labels))
            }

    return (torch.stack(train_images), torch.tensor(train_labels), 
            torch.stack(test_images), torch.tensor(test_labels),
            participant_data)

In [20]:
class MixedDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        if isinstance(self.dataset, TensorDataset):
            x, y = self.dataset[index]
        else:
            x, y = self.dataset[index]
        
        # Ensure x is a tensor
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x)
        
        # Ensure y is a tensor
        if not isinstance(y, torch.Tensor):
            y = torch.tensor(y)
        
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.dataset)

In [21]:
# Model Definition
class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(16 * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Training function (slightly modified)
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience):
    model = model.to(device)
    best_val_acc = 0
    epochs_no_improve = 0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        print(f"Starting epoch {epoch+1}")
        for i, (images, labels) in enumerate(train_loader):
            print(f"Batch {i+1}/{len(train_loader)}")
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()
            if i % 10 == 0:
                print(f"Epoch {epoch+1}, Batch {i+1}, Loss: {loss.item():.4f}")
        
        print(f"Finished epoch {epoch+1}")
        
        # Validation
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = model(val_images)
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        val_acc = 100 * correct_val / total_val
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        
        if epochs_no_improve == patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break
    
    return model

In [22]:
def k_fold_cross_validation(dataset, num_folds=3, num_epochs=30, patience=5):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_results = []
    
    for fold, (train_index, val_index) in enumerate(kf.split(range(len(dataset)))):
        print(f"Fold {fold + 1}/{num_folds}")
        train_subset = torch.utils.data.Subset(dataset, train_index)
        val_subset = torch.utils.data.Subset(dataset, val_index)
        print(f"Train subset size: {len(train_subset)}, Validation subset size: {len(val_subset)}")

        train_loader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=0, pin_memory=True, prefetch_factor=2)
        val_loader = DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=0, pin_memory=True, prefetch_factor=2)
        
        print("Creating model")

        model = LeNet5_16x16(num_classes=10).to(device)
        print("Calculating class weights")

        labels = torch.tensor([dataset[i][1] for i in train_index])
        class_counts = torch.bincount(labels, minlength=10)
        class_weights = 1. / class_counts.float()
        class_weights = class_weights / class_weights.sum()
        criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
        
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        print("Starting training for this fold")
        start_time = time.time()
        model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience)
        print(f"Training time for fold {fold + 1}: {time.time() - start_time:.2f} seconds")
        
        # Evaluate on validation set
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                outputs_val = model(val_images)
                _, predicted_val = torch.max(outputs_val.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted_val == val_labels).sum().item()
        val_accuracy = 100 * correct_val / total_val
        fold_results.append(val_accuracy)
        print(f'Fold {fold + 1} Validation Accuracy: {val_accuracy:.2f}%')
    
    return fold_results, model

This did not work, it crashed after the first model, we are going to create a light version with 3 folds and 30 epochs each to train the 25 models into a specific folder.

In [23]:
def test_data_loading(dataset):
    print("Testing data loading")
    test_loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
    start_time = time.time()
    for i, (images, labels) in enumerate(test_loader):
        if i % 100 == 0:
            print(f"Batch {i+1}: Image shape: {images.shape}, Label shape: {labels.shape}")
        if i == 500:  # Test with 500 batches
            break
    print(f"Data loading test completed in {time.time() - start_time:.2f} seconds")

In [24]:
# Data augmentation (slightly modified)
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
    
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

class AugmentedMNISTDataset(Dataset):
    def __init__(self, is_train=True, invert=False):
        self.dataset = datasets.MNIST(root='./data', train=is_train, download=True)
        self.invert = invert
        self.transform = transforms.Compose([
            transforms.Resize((16, 16)),
            transforms.ToTensor(),
            transforms.RandomApply([
                transforms.RandomRotation(15),
                transforms.RandomAffine(0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                AddGaussianNoise(0., 0.05),
            ], p=0.7),
            transforms.RandomApply([
                transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0),
            ], p=0.3),
        ])

    def __getitem__(self, index):
        img, label = self.dataset[index]
        img = self.transform(img)
        if self.invert:
            img = 1 - img
        return img, label

    def __len__(self):
        return len(self.dataset)

# def load_augmented_mnist():
#     mnist_train = AugmentedMNISTDataset(is_train=True, invert=False)
#     mnist_test = AugmentedMNISTDataset(is_train=False, invert=False)
#     inverted_mnist_train = AugmentedMNISTDataset(is_train=True, invert=True)
#     inverted_mnist_test = AugmentedMNISTDataset(is_train=False, invert=True)
    
#     combined_train = ConcatDataset([mnist_train, inverted_mnist_train])
#     combined_test = ConcatDataset([mnist_test, inverted_mnist_test])
    
#     return combined_train, combined_test

def load_augmented_mnist():
    transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
    ])
    
    mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    
    return mnist_train, mnist_test

def train_ensemble(num_models=25, num_folds=3, num_epochs=30, patience=5):
    ensemble = []
    os.makedirs('lenet5_ensemble', exist_ok=True)
    
    for i in range(num_models):
        print(f"Training model {i+1}/{num_models}")
        
        # Load experimental data
        start_time = time.time()
        exp_train_images, exp_train_labels, exp_test_images, exp_test_labels, participant_data = load_all_experimental_data('test_digits')
        print(f"Experimental data loading time: {time.time() - start_time:.2f} seconds")
        
        exp_dataset = TensorDataset(exp_train_images, exp_train_labels)
        
        # Load augmented MNIST data
        start_time = time.time()
        mnist_train, mnist_test = load_augmented_mnist()
        print(f"MNIST data loading time: {time.time() - start_time:.2f} seconds")
        
        # Combine experimental data with augmented MNIST data
        combined_dataset = ConcatDataset([MixedDataset(exp_dataset), mnist_train])
        test_data_loading(combined_dataset)
        print(f"Total dataset size: {len(combined_dataset)}")
        # Perform K-Fold Cross Validation
        start_time = time.time()
        fold_results, model = k_fold_cross_validation(combined_dataset, num_folds=num_folds, num_epochs=num_epochs, patience=patience)
        print(f"K-Fold Cross Validation time: {time.time() - start_time:.2f} seconds")
        print(f"Model {i+1} - Average Validation Accuracy across folds: {np.mean(fold_results):.2f}%")
        
        ensemble.append(model)
        
        # Save the model
        torch.save(model.state_dict(), f"lenet5_ensemble/lenet5_trained_model_ensemble_{i+1}.pth")
        print(f"Model {i+1} saved as lenet5_ensemble/lenet5_trained_model_ensemble_{i+1}.pth")
    
    return ensemble

def ensemble_predict(ensemble, input_tensor):
    predictions = []
    for model in ensemble:
        model.eval()
        with torch.no_grad():
            output = model(input_tensor)
        predictions.append(torch.softmax(output, dim=1))
    avg_prediction = torch.mean(torch.stack(predictions), dim=0)
    return torch.argmax(avg_prediction, dim=1)

if __name__ == "__main__":
    ensemble = train_ensemble(num_models=25, num_folds=3, num_epochs=30, patience=5)
    
    _, _, exp_test_images, exp_test_labels, _ = load_all_experimental_data('test_digits')
    exp_test_dataset = TensorDataset(exp_test_images, exp_test_labels)
    exp_test_loader = DataLoader(exp_test_dataset, batch_size=32, num_workers=0, pin_memory=True, shuffle=False)
    
    correct = 0
    total = 0
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
        with record_function("model_training"):
            for images, labels in exp_test_loader:
                nvtx.range_push("Training Loop")
                images, labels = images.to(device), labels.to(device)
                predictions = ensemble_predict(ensemble, images)
                total += labels.size(0)
                correct += (predictions == labels).sum().item()
                accuracy = 100 * correct / total
                nvtx.range_pop()
    
    print(f"Ensemble accuracy on experimental data: {accuracy:.2f}%")
    print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

Training model 1/25
Experimental data loading time: 8.74 seconds
MNIST data loading time: 0.05 seconds
Testing data loading


RuntimeError: DataLoader worker (pid(s) 12852, 9488, 38236, 24152) exited unexpectedly